<h1>Aflevering 7</h1>
<h3>Hold CE2</h3>
Gruppe: Alexander Bjerno & Christian Winkler

In [2]:
import numpy as np
import matplotlib.pyplot as plt


<h3>(a)</h3>

En parkeringsplads på 20 m × 30 m bliver oplyst via lamper placeret forskel-
lige steder og i forskellige højde, som angivet i figur 1 side 3.
Parkeringspladsen inddeles i en rektangulær gitter af 600 kvadrater hver
af størrelse 1 m ×1 m . Tallet 𝑦𝑗 angiver belysningsniveauet i kvadrat 𝑗, for
𝑗 =0,...,599. Lad 𝑥𝑖 angiver styrken af lampe 𝑖. Vi vælger enheder så at
bidraget til belysningen i kvadrat 𝑗 fra lampe 𝑖 er 𝑥𝑖/𝑑2𝑖𝑗, hvor 𝑑𝑖𝑗 er afstanden i
ℝ3 fra lampen til centrum af kvadrat 𝑗.
(a) Angiv hvordan belysningsniveauet 𝑦 =(𝑦0,...,𝑦599) og styrkerne 𝑥 =
(𝑥0,...,𝑥11) er relateret via et lineært ligningssystem. Opstil koefficientmatricen for systemet i python. (I må estimere koordinaterne for
placeringen af hver lampe ud fra diagrammet.


In [ ]:
import numpy as np

# Estimate lamp coordinates from diagram
lamp_coords = np.array([
    [2., 3., 3.], [4., 19., 3.], [4., 13., 3.6], [11., 6., 3.5], [15., 2., 4.5], [20., 5., 2.8],
    [28., 9., 3.4], [12., 12., 4.], [23., 12., 4.], [16., 16., 3.], [25., 16., 3.8], [13., 18., 3.6]
])

# Calculate distance between each lamp and each square center
square_coords = np.array([(i, j) for i in range(20) for j in range(30)]) + 0.5
distances = np.linalg.norm(
    lamp_coords[:, np.newaxis, :] - square_coords[np.newaxis, :, :], axis=-1)

# Calculate A matrix
A = np.divide(lamp_coords[:, 2][:, np.newaxis], distances)
